In [0]:
%run ../Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets

In [0]:
files = dbutils.fs.ls(f"{dataset_bookstore}/orders-raw")
display(files)

In [0]:
( spark.readStream
  .format('cloudFiles')
  .option('cloudFiles.format', 'parquet')
  .option('cloudFiles.schemaLocation', "dbfs:/mnt/checkpoints/orders_raw")
  .load(f"{dataset_bookstore}/orders-raw")
  .createOrReplaceTempView('orders_raw_temp')
)

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW orders_tmp AS(
  SELECT *, current_timestamp() as arrival_time, input_file_name() as source_file
  FROM orders_raw_temp
)

In [0]:
%sql
select * from orders_tmp


In [0]:
(spark.table('orders_tmp')
  .writeStream
  .format('delta')
  .option('checkpointLocation', 'dbfs:/mnt/demo/checkpoints/orders_bronze')
  .outputMode("append")
  .table('orders_bronze')

)

In [0]:
%sql
select count(*) from orders_bronze

In [0]:
load_new_data()

In [0]:
%sql
select count(*) from orders_bronze

--Here are the DataFrames in the session

In [0]:
(spark.read
 .format('json')
 .load(f"{dataset_bookstore}/customers-json")
 .createOrReplaceTempView('customers_lookup')
)

In [0]:
%sql
select * from customers_lookup

In [0]:
(
spark.readStream
.table("orders_bronze")
.createOrReplaceTempView("orders_bronze_temp")
                     
)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW ordered_enriched_tmp AS
( SELECT order_id, quantity, o.customer_id, c.profile:first_name as f_name, c.profile:last_name as l_name,
    cast(from_unixtime(order_timestamp, "yyyy-MM-dd HH:mm:ss") as timestamp) order_timestamp, books

FROM orders_bronze_temp o
INNER JOIN customers_lookup c
ON o.customer_id = c.customer_id
WHERE quantity > 0




)

In [0]:
(spark.table("ordered_enriched_tmp")
    .writeStream
    .format("delta")
    .option("checkpointLocation", "dbfs:/mnt/demo/checkpoints/orders_silver")
    .outputMode("append")
    .table("orders_silver")



)

In [0]:
%sql
select * from orders_silver

In [0]:
%sql
select count(*) from orders_silver

In [0]:
for s in spark.streams.active:
  print(f"stopped stream: {s.name}")
  s.stop()
  s.awaitTermination()